In [1]:
PROJECT_NAME = "wheel_mixtral_8X7B"

In [2]:
from langchain import LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.llms import Replicate
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
replicate_api_key = os.environ["REPLICATE_API_TOKEN"]
groq_api_key = os.environ["GROQ_API_TOKEN"]

template = """You are a helpful assistant for creating a .geo file to be used in multiphysics simulations with software like ELMER or Gmsh. Your task is to analyze the geometry the user wants and ask questions about all required dimensions until you have all the data to produce the file. In the file, define physical groups for easy identification of different parts later on in the simulation software. Follow these sections meticulously:
1.	Start with OpenCASCADE Geometry Kernel
•	Begin the file by switching to the OpenCASCADE geometry kernel with
// Switch to OpenCASCADE geometry kernel
SetFactory("OpenCASCADE");
•	Remember, OpenCASCADE primitives automatically create their defining points.
2.	Define the Dimensions
•	Start by defining any necessary variables that will be used throughout the file.
•	Example: L = 10; // Length in cm.
3.	Define the Points
•	Define the points that make up the geometry. Each point should be defined with its coordinates in 3D space and a characteristic length.
•	Example: Point(1) = {{x, y, z, lc}}; where x, y, z are coordinates, and lc is the characteristic length.
4.	Define Lines
•	Create lines by joining the defined points. Each line connects two points.
•	Example: Line(1) = {{Point1, Point2}};
5.	Define Circles
•	Create circles by defining three points: the start point, center point, and end point.
•	Example: Circle(1) = {{Point1, CenterPoint, Point2}};
6.	Define Cylinders
•	Use cylinders for simple cylindrical shapes.
•	Example: Cylinder(1) = {{x1, y1, z1, Dx, Dy, Dz, r}}; where x1, y1, z1 define the base center of the cylinder, Dx, Dy, Dz represent the relative displacement from the base center to the top center, and r is the radius of the cylinder.
7.	Define Spheres
•	Use spheres for simple spherical shapes.
•	Example: Sphere(1) = {{x, y, z, r}}; where x, y, z define the center and r is the radius.
8.	Define Boxes
•	Use boxes for simple rectangular or cubic shapes.
•	Example: Box(1) = {{x1, y1, z1, dx, dy, dz}}; where x1, y1, z1 define the corner and dx, dy, dz define the dimensions.
9.	Define Cones
•	Use cones for simple conical shapes.
•	Example: Cone(1) = {{x1, y1, z1, r1, x2, y2, z2, r2}}; where x1, y1, z1, r1 define the base and x2, y2, z2, r2 define the top.
10.	Define Extrusions and Revolutions
•	Use extrusions or revolutions for shapes that can be formed by extending or rotating a 2D profile.
•	Extrusion Example: Extrude {{dx, dy, dz}} {{Surface{{1}}; Layers{{N}}; Recombine;}};
•	Revolution Example: Rotate {{Axis{{ax, ay, az}}, Point{{px, py, pz}}, Angle{{theta}};}} {{Surface{{1}}; Layers{{N}}; Recombine;}};
11.	Define the Surfaces
•	Surfaces are defined based on the lines or curves. Start by creating a Line Loop which is a closed loop of lines.
•	Example: Line Loop(LoopID) = {{Line1, Line2, ..., LineN}};
•	Then, define a surface. This can be a Plane Surface for flat surfaces or a Ruled Surface for curved surfaces.
•	Example: Plane Surface(SurfaceID) = {{LoopID}}; or Ruled Surface(SurfaceID) = {{LoopID}};
12.	Define the Volume
•	For 3D geometries, define volumes. Start by creating a Surface Loop which is a collection of surfaces that enclose a volume.
•	Example: Surface Loop(SurfaceLoopID) = {{Surface1, Surface2, ..., SurfaceN}};
•	Then, define a volume enclosed by this surface loop.
•	Example: Volume(VolumeID) = {{SurfaceLoopID}};
13.	Define Physical Groups for Boundaries
•	Assign meaningful names to different parts of the geometry using physical groups. This is crucial for identifying boundaries and regions during simulation.
•	Example for surfaces: Physical Surface("Name") = {{SurfaceID}};
•	Example for volume: Physical Volume("Name") = {{VolumeID}};
•	Use descriptive names like "Bottom", "Top", "Side1", etc., for surfaces and an appropriate name for the volume.
Normally geometries do not require all these sections to be defined. Always aim for the simplest approach to model the geometry. If there are multiple ways to create a shape, choose the simplest one.  
Additional Instructions:
•	Ensure that the identifiers (like Point1, Line1, SurfaceID, etc.) are unique and correctly referenced. Numbering starts with 1.
•	Check that all points used in Line, Circle, etc., do exist in the Point list.
•	Include comments for clarity, using // to start a comment line.
•	Pay attention to the orientation of lines and surfaces, as this affects the mesh generation in Gmsh.
•	Verify the geometric consistency: Ensure that the dimensions, positions, and connections of elements are logical and physically plausible.
•	In doubt, ask the user for clarification.
•	If the user provides information that is not relevant or tries to change the subject, steer the conversation back to the original focus of providing information for the .geo file.
Output Instructions:
•	To facilitate the extraction of this text by a subsequent function, please enclose the entire .geo file content within specific markers:
•	Start the file content with //BEGIN_GEO.
•	End the file content with //END_GEO.
•	Ensure that the text between these markers is exactly as it should appear in the .geo file, with correct syntax and formatting.
Example:
//BEGIN_GEO
// Switch to OpenCASCADE geometry kernel
SetFactory("OpenCASCADE");
[Insert the complete and final .geo file content here]
//END_GEO

{history}
Human: {human_input}
Assistant:"""

prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)     

In [3]:
#Saves .geo file from response
def extract_and_save_geo_file(response_text,PROJECT_NAME=PROJECT_NAME): 
    file_path = f"{PROJECT_NAME}.geo"
    try:
        # Use string formatting explained in system_geo.txt 
        start_marker = "//BEGIN_GEO"
        end_marker = f"//END_GEO"

        start_index = response_text.index(start_marker) + len(start_marker)
        end_index = response_text.index(end_marker, start_index)
        geo_content = response_text[start_index:end_index].strip()

        with open(file_path, 'w') as file:
            file.write(geo_content)
        print(f"{file_path} file saved")
        print("Visualize .geo files with GMSH")

    except ValueError as e:
        print(f"ERROR: .geo file NOT SAVED.")

In [4]:
#FUNCTIONS FROM CREATING THE MESH ONCE THE .GEO FILE
import os
import gmsh

def generate_mesh(Mesh_Min_Size, Mesh_Max_Size):
    print("Creating MESH...\n ")
    try:
        current_directory = os.getcwd()
        input_geo_file = os.path.join(current_directory, f'{PROJECT_NAME}.geo') 
        output_msh_file = os.path.join(current_directory, f'{PROJECT_NAME}.msh')  
        # Initialize gmsh
        gmsh.initialize()
        # Open the .geo file
        gmsh.open(input_geo_file)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMin", Mesh_Min_Size)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMax", Mesh_Max_Size)
        # Generate the mesh using default settings
        gmsh.model.mesh.generate(3)
        # Save the mesh to a .msh file
        gmsh.write(output_msh_file)
        # Finalize gmsh
        gmsh.finalize()
        print(f"{PROJECT_NAME}.msh saved")
        print("You can visualize .msh files with GMSH, Paraview or, within this notebook, with plot_grid()")      
    except ValueError as e:
        print(f"ERROR: MESH NOT CREATED.")
        
#(optional)
import pyvista as pv

def plot_grid(output_msh_file):
    # Read mesh file
    mesh = pv.read(output_msh_file)
    # Set the Jupyter backend to 'static'
    pv.set_jupyter_backend('static')
    # Plot the mesh
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(mesh, show_edges=True)
    plotter.show()

In [5]:
mixtral = ChatGroq(temperature=0,groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")

In [6]:
chain = LLMChain(
    llm=mixtral,
    prompt=prompt,
    verbose=False,
    memory=ConversationBufferWindowMemory(k=8),
    llm_kwargs={"max_tokens":4096,"temperature": 0.00}
)

C:\Users\ossama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
output = chain.predict(
    human_input="Create a wheel and axle. Keep it simple. Assume that both Wheels and axle are cylinders.The wheels are located at the two ends of the axle. The axis of all cylinders is parallel to the z axis")
print(output)

Sure, I can help you define a simple wheel and axle system using the geometry description language you provided. Here's a basic .geo file:

//BEGIN_GEO
// Switch to OpenCASCADE geometry kernel
SetFactory("OpenCASCADE");

// Define the dimensions
R = 0.15; // Radius of the wheels and axle in meters
L = 0.3; // Length of the axle in meters

// Define the Points
Point(1) = {0, 0, 0, R};
Point(2) = {0, 0, L, R};
Point(3) = {L, 0, L, R};
Point(4) = {L, 0, 0, R};

// Define the Lines
Line(1) = {1, 2};
Line(2) = {2, 3};
Line(3) = {3, 4};
Line(4) = {4, 1};

// Define the Circles (which are actually Cylinders in 3D)
Cylinder(1) = {1, 1, 0, 2*R, 0, 0, R}; // Wheel 1
Cylinder(2) = {4, 4, 0, 2*R, 0, 0, R}; // Wheel 2
Cylinder(3) = {2, 3, L, 0, 0, 0, R}; // Axle

// Define the Surfaces
Plane Surface(1) = {1};
Plane Surface(2) = {2};
Plane Surface(3) = {3};
Plane Surface(4) = {4};

// Define the Volume
Volume(1) = {1, 2, 3, 4};

// Define Physical Groups for Boundaries
Physical Surface("Wheel1") = {

In [8]:
output = chain.predict(
    human_input="""1. Radius of the wheel (Rw): 5 cm,
Width of the wheel (Ww): 2 cm,
Radius of the axle (Ra): 1 cm,
Length of the axle (La): 20 cm""")
print(output)

Sure, I can modify the dimensions based on your provided values. Here's the updated .geo file:

//BEGIN_GEO
// Switch to OpenCASCADE geometry kernel
SetFactory("OpenCASCADE");

// Define the Dimensions
Rw = 0.05; // Radius of the wheel in meters
Ww = 0.02; // Width of the wheel in meters
Ra = 0.01; // Radius of the axle in meters
La = 0.2; // Length of the axle in meters

// Define the Points
Point(1) = {0, 0, 0, Rw};
Point(2) = {0, Ww/2, La, Rw};
Point(3) = {Ww, Ww/2, La, Rw};
Point(4) = {Ww, 0, La, Rw};
Point(5) = {0, 0, La, Ra};
Point(6) = {0, Ww/2, La, Ra};
Point(7) = {Ww, Ww/2, La, Ra};
Point(8) = {Ww, 0, La, Ra};

// Define the Lines
Line(1) = {1, 2};
Line(2) = {2, 3};
Line(3) = {3, 4};
Line(4) = {4, 1};
Line(5) = {5, 6};
Line(6) = {6, 7};
Line(7) = {7, 8};
Line(8) = {8, 5};

// Define the Circles (which are actually Cylinders in 3D)
Cylinder(1) = {1, 2, 0, 2*Rw, 0, 0, Rw}; // Wheel 1
Cylinder(2) = {4, 3, 0, 2*Rw, 0, 0, Rw}; // Wheel 2
Cylinder(3) = {5, 6, La, 0, 0, 0, Ra}; // Ax

In [10]:
output = chain.predict(
    human_input="i want the dimensions in cm instead of m")
print(output)

Sure, I can convert the units to centimeters for you. Here's the updated .geo file:

//BEGIN_GEO
// Switch to OpenCASCADE geometry kernel
SetFactory("OpenCASCADE");

// Define the Dimensions
Rw = 5; // Radius of the wheel in cm
Ww = 2; // Width of the wheel in cm
Ra = 1; // Radius of the axle in cm
La = 20; // Length of the axle in cm

// Define the Points
Point(1) = {0, 0, 0, Rw};
Point(2) = {0, Ww/2, La, Rw};
Point(3) = {Ww, Ww/2, La, Rw};
Point(4) = {Ww, 0, La, Rw};
Point(5) = {0, 0, La, Ra};
Point(6) = {0, Ww/2, La, Ra};
Point(7) = {Ww, Ww/2, La, Ra};
Point(8) = {Ww, 0, La, Ra};

// Define the Lines
Line(1) = {1, 2};
Line(2) = {2, 3};
Line(3) = {3, 4};
Line(4) = {4, 1};
Line(5) = {5, 6};
Line(6) = {6, 7};
Line(7) = {7, 8};
Line(8) = {8, 5};

// Define the Circles (which are actually Cylinders in 3D)
Cylinder(1) = {1, 2, 0, 2*Rw, 0, 0, Rw}; // Wheel 1
Cylinder(2) = {4, 3, 0, 2*Rw, 0, 0, Rw}; // Wheel 2
Cylinder(3) = {5, 6, La, 0, 0, 0, Ra}; // Axle

// Define the Surfaces
Plane Surf

In [11]:
extract_and_save_geo_file(output)

wheel_mixtral_8X7B.geo file saved
Visualize .geo files with GMSH


In [12]:
output = chain.predict(
    human_input="The centre of the wheels should be aligned axially")
print(output)

I apologize for the misunderstanding. I've updated the .geo file to align the centers of the wheels axially. Here's the corrected version:

//BEGIN_GEO
// Switch to OpenCASCADE geometry kernel
SetFactory("OpenCASCADE");

// Define the Dimensions
Rw = 5; // Radius of the wheel in cm
Ww = 2; // Width of the wheel in cm
Ra = 1; // Radius of the axle in cm
La = 20; // Length of the axle in cm

// Define the Points
Point(1) = {0, 0, 0, Rw};
Point(2) = {0, Ww/2, La/2, Rw};
Point(3) = {Ww, Ww/2, La/2, Rw};
Point(4) = {Ww, 0, La/2, Rw};
Point(5) = {0, 0, La/2, Ra};
Point(6) = {0, Ww/2, La/2, Ra};
Point(7) = {Ww, Ww/2, La/2, Ra};
Point(8) = {Ww, 0, La/2, Ra};

// Define the Lines
Line(1) = {1, 2};
Line(2) = {2, 3};
Line(3) = {3, 4};
Line(4) = {4, 1};
Line(5) = {5, 6};
Line(6) = {6, 7};
Line(7) = {7, 8};
Line(8) = {8, 5};

// Define the Circles (which are actually Cylinders in 3D)
Cylinder(1) = {1, 2, 0, 2*Rw, 0, 0, Rw}; // Wheel 1
Cylinder(2) = {4, 3, 0, 2*Rw, 0, 0, Rw}; // Wheel 2
Cylinder(3) 

In [13]:
extract_and_save_geo_file(output,PROJECT_NAME="wheel_mixtral_8X7B_updated")

wheel_mixtral_8X7B_updated.geo file saved
Visualize .geo files with GMSH


In [ ]:
generate_mesh(Mesh_Min_Size = 0.05, Mesh_Max_Size = 0.1)

In [ ]:
# it is not combined into one physical volume the two wheel vols should be deleted 